# Collect & clean data

This notebook is dedicated to collecting, cleaning and analyzing all the data that we need for our story. This includes:
- Historical water level data for the Edwards Aquifer.
- Historical fullness data for several nearby lakes.
- Historical discharge data for several nearby rivers.

I will list out the sources of the data as we go along.

* [Imports](#Imports)
* [Edwards Aquifer water levels](#Edwards-Aquifer-water-levels)

## Imports

Let's start our project by importing all the libraries we need.

In [310]:
# To automate the collection of certain datasets, we'll use beautifulsoup4 and requests
import requests
from bs4 import BeautifulSoup

# For data manipulation
import pandas as pd

# We use this library to streamline the process of downloading data from the USGS
import dataretrieval.nwis as nwis

## Edwards Aquifer water levels

The first dataset we'll collect and clean is the Edwards Aquifer Authority's J17 aquifer data [from their historical data page](https://www.edwardsaquifer.org/science-maps/aquifer-data/historical-data/).

In [311]:
edwards_aquifer_authority_url = 'https://www.edwardsaquifer.org/science-maps/aquifer-data/historical-data/'

def getSoup(url):
    page = requests.get(url) 
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

soup = getSoup(edwards_aquifer_authority_url)

# Find the href that contains the word 'csv'
csv_links = soup.find_all('a', href=lambda href: href and 'csv' in href)
j17_historical_data = csv_links[0]['href']

# Import j17_historical_data into a pandas dataframe
j17_historical_data_df = pd.read_csv(j17_historical_data)

j17_historical_data_df.head()

,Site,DailyHighDate,WaterLevelElevation
0,J17WL,2023-03-02,635.55
1,J17WL,2023-03-01,635.98
2,J17WL,2023-02-28,636.29
3,J17WL,2023-02-27,636.80
4,J17WL,2023-02-26,637.31


Alright, we successfully used beautiful soup to grab the J17 data off of the historical data page and imported it into a pandas dataframe. Now we'll clean it up a bit.

In [312]:
# Let's convert the "DailyHighDate" column to a datetime object
j17_historical_data_df['DailyHighDate'] = pd.to_datetime(j17_historical_data_df['DailyHighDate'])

# Let's create a new column titled "Year" that contains the year of the "DailyHighDate" column
j17_historical_data_df['Year'] = j17_historical_data_df['DailyHighDate'].dt.year

# Let's create a new column titled "dw_date" that duplicates the data found in the "DailyHighDate" column, but replaces the year with 2050. We're doing all of this to make it easier for us to visualize it in a Datawrapper line chart in the future. The 2050 year will not appear in the final product and has no bearing on our analysis.
j17_historical_data_df['dw_date'] = j17_historical_data_df['DailyHighDate'].dt.strftime('%m/%d') + '/2050'

# Let's filter the dataframe to only include years from 1980 to 2023.
j17_historical_data_df = j17_historical_data_df[(j17_historical_data_df['Year'] >= 2000) & (j17_historical_data_df['Year'] <= 2023)].reset_index()

# Find the value in the "dw_date" column that is in the same row as the max value in the "date" column and print the first one out
latest_date = j17_historical_data_df[j17_historical_data_df['DailyHighDate'] == j17_historical_data_df['DailyHighDate'].max()]['dw_date'].values[0]

# Let's filter the dataframe to only include rows where the dw_date is between 2023-01-01 and 2023-03-02
j17_historical_data_df = j17_historical_data_df[(j17_historical_data_df['dw_date'] >= '01/01/2023') & (j17_historical_data_df['dw_date'] <= latest_date)].reset_index()

# Pivot the j17_historical_data_df so that each year is a column and dw_date is the index with the values being the "DailyHigh" column. This is so that we can easily visualize it in a Datawrapper line chart.
j17_historical_data_df = j17_historical_data_df.pivot(index='dw_date', columns='Year', values='WaterLevelElevation').reset_index()

# Export j17_historical_data_df to a csv file
j17_historical_data_df.to_csv('../output/aquifers/j17_historical_data.csv', index=False)

# Let's take a look at the first 5 rows of the dataframe
j17_historical_data_df.head()

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,01/01/2050,663.46,676.72,684.80,694.75,679.44,697.33,NaN,666.10,689.19,...,640.07,633.79,666.70,685.35,666.10,686.42,671.94,663.78,663.84,637.61
1,01/02/2050,663.90,676.59,684.68,694.64,679.44,697.29,NaN,666.00,689.10,...,640.69,634.04,666.37,685.41,665.94,686.20,672.13,663.88,663.63,637.40
2,01/03/2050,663.92,676.38,684.40,694.41,679.42,697.02,677.49,665.84,688.98,...,640.87,634.41,666.69,685.51,665.62,686.32,671.81,664.03,663.25,637.16
3,01/04/2050,663.56,676.40,684.22,694.35,679.27,697.03,677.13,666.96,688.88,...,640.99,634.73,667.70,685.12,665.09,686.50,671.69,663.86,663.02,636.72
4,01/05/2050,663.44,676.45,684.51,694.43,678.96,697.04,677.09,667.88,688.99,...,641.71,634.93,667.55,684.81,664.44,686.72,671.66,663.82,662.89,636.41


## The lakes

Now that we have the Edwards Aquifer data, we'll grab the water level data for several nearby lakes. The lakes we'll be using are:
- Medina Lake
- Canyon Lake
- Lake Travis

We're sourcing this data from [waterdatafortexas.org](https://waterdatafortexas.org/reservoirs/statewide), a product from [the Texas Water Development Board](https://www.twdb.texas.gov/). Their mission "is to lead the state's efforts in ensuring a secure water future for Texas and its citizens."

In [313]:
# Let's import the lake data from the waterdatafortexas.org website. We don't need to use BeautifulSoup for this because the data appears to be located at a static URL. We skip rows because there's a bunch of metadata at the top of the CSV file that we don't need.
medina_lake_full_data_df = pd.read_csv("https://www.waterdatafortexas.org/reservoirs/individual/medina.csv", skiprows=55)
canyon_lake_full_data_df = pd.read_csv("https://www.waterdatafortexas.org/reservoirs/individual/canyon.csv", skiprows=54)
travis_lake_full_data_df = pd.read_csv("https://www.waterdatafortexas.org/reservoirs/individual/travis.csv", skiprows=53)

# I'll print out the head of medina_lake_full_data_df so that we can see what it looks like ahead of time.
travis_lake_full_data_df.head()

,date,water_level,surface_area,reservoir_storage,conservation_storage,percent_full,conservation_capacity,dead_pool_capacity
0,1940-09-30,550.7,1864.0,44232,22807,2.0,1113531,21425
1,1940-10-31,548.2,1750.0,39722,18297,1.6,1113531,21425
2,1940-11-30,595.5,5209.0,194535,173110,15.5,1113531,21425
3,1940-12-31,615.0,7320.0,315950,294525,26.4,1113531,21425
4,1941-01-31,614.0,7205.0,308688,287263,25.8,1113531,21425


In [314]:
# I want to have all the lakes in a single dataframe so that we can easily clean all of the data at once. I'll create a new dataframe called "lake_data_df" that contains all of the data from the other dataframes. Before exporting the data to a CSV file, I'll break it up into individual dataframes again.

# Before we do that, let's create a new column called "Lake" that contains the name of the lake. We'll use this column to filter the data later on.
medina_lake_full_data_df['Lake'] = 'Medina'
canyon_lake_full_data_df['Lake'] = 'Canyon'
travis_lake_full_data_df['Lake'] = 'Travis'

# Let's create a new dataframe called "lake_data_df" that contains all of the data from the other dataframes.
lake_data_df = pd.concat([medina_lake_full_data_df, canyon_lake_full_data_df, travis_lake_full_data_df])

# Let's take a look at the first 5 rows of the dataframe
lake_data_df.head()

,date,water_level,surface_area,reservoir_storage,conservation_storage,percent_full,conservation_capacity,dead_pool_capacity,Lake
0,1997-08-09,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
1,1997-08-10,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
2,1997-08-11,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
3,1997-08-12,1072.0,6662.23,304449,254823,100.0,254823,0,Medina
4,1997-08-13,1071.9,6653.70,303783,254823,100.0,254823,0,Medina


In [315]:
# Alright, let's do some cleaning.

# Let's convert the date column of each dataframe to a datetime object
lake_data_df['date'] = pd.to_datetime(lake_data_df['date'])

# Let's create a new column titled "Year" that contains the year of the "date" column
# medina_lake_full_data_df['Year'] = medina_lake_full_data_df['date'].dt.year
lake_data_df['Year'] = lake_data_df['date'].dt.year

# Let's create a new column titled "dw_date" that duplicates the data found in the "date" column, but replaces the year with 2050. We're doing all of this to make it easier for us to visualize it in a Datawrapper line chart in the future. The 2050 year will not appear in the final product and has no bearing on our analysis.
lake_data_df['dw_date'] = lake_data_df['date'].dt.strftime('%m/%d') + '/2050'

# Find the value in the "dw_date" column that is in the same row as the max value in the "date" column and print the first one out
latest_date = lake_data_df[lake_data_df['date'] == lake_data_df['date'].max()]['dw_date'].iloc[0]

# Let's only keep records where the "Year" column in the lake_data_df is greater than or equal to 2000
lake_data_df = lake_data_df[lake_data_df['Year'] >= 2000]

# Let's filter the dataframe to only include rows where the dw_date is between 2050-01-01 and 2050-03-20
lake_data_df = lake_data_df[(lake_data_df['dw_date'] >= '01/01/2050') & (lake_data_df['dw_date'] <= latest_date)].reset_index()

# Let's break up the lake_data_df dataframe into individual dataframes again.
medina_lake_full_data_df = lake_data_df[lake_data_df['Lake'] == 'Medina']
canyon_lake_full_data_df = lake_data_df[lake_data_df['Lake'] == 'Canyon']
travis_lake_full_data_df = lake_data_df[lake_data_df['Lake'] == 'Travis']

# Pivot the medina_lake_full_data_df so that each year is a column and dw_date is the index with the values being the "percent_full" column. This is so that we can easily visualize it in a Datawrapper line chart.
medina_lake_full_data_df = medina_lake_full_data_df.pivot(index='dw_date', columns='Year', values='percent_full').reset_index()
canyon_lake_full_data_df = canyon_lake_full_data_df.pivot(index='dw_date', columns='Year', values='percent_full').reset_index()
travis_lake_full_data_df = travis_lake_full_data_df.pivot(index='dw_date', columns='Year', values='percent_full').reset_index()

# Export each of the dataframes to a csv file
medina_lake_full_data_df.to_csv('../output/lakes/medina_lake_full_data.csv', index=False)
canyon_lake_full_data_df.to_csv('../output/lakes/canyon_lake_full_data.csv', index=False)
travis_lake_full_data_df.to_csv('../output/lakes/travis_lake_full_data.csv', index=False)

canyon_lake_full_data_df.head()

Year,dw_date,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,01/01/2050,93.4,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,84.4,75.5,100.0,100.0,92.7,100.0,93.4,89.0,99.4,79.6
1,01/02/2050,93.3,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,84.4,75.5,100.0,100.0,92.7,100.0,93.4,88.9,99.4,79.6
2,01/03/2050,93.4,100.0,100.0,100.0,99.7,100.0,95.2,85.0,100.0,...,84.3,75.6,100.0,100.0,92.7,100.0,93.4,88.9,99.3,79.6
3,01/04/2050,93.4,100.0,100.0,100.0,99.8,100.0,95.2,85.1,100.0,...,84.3,75.6,100.0,100.0,92.6,100.0,93.4,88.9,99.2,79.5
4,01/05/2050,93.3,100.0,100.0,99.7,99.8,100.0,95.2,85.1,100.0,...,84.3,75.5,100.0,100.0,92.6,100.0,93.3,88.9,99.2,79.4


## The rivers

Now that we have the Edwards Aquifer data and the lake data, we'll grab the discharge data for several nearby rivers.

The data are coming from the USGS [National Water Information System](https://waterdata.usgs.gov/nwis/rt) (NWIS) through their [dataretrieval-python library](https://github.com/DOI-USGS/dataretrieval-python).

 The rivers we'll be using are:
- Guadalupe River at Spring Branch, Texas
- Guadalupe River at Sattler, Texas
- Guadalupe River at New Braunfels, Texas
- San Marcos River in San Marcos, Texas
- Blanco River at Wimberley, Texas
- Medina River at Patterson Road in Medina, Texas
- Medina River at La Coste, Texas
- San Antonio River near Floresvilles, Texas

In [316]:
# Let's start by creating a dictionary that contains the name of the rivers we're interested in and its site number. We'll use this dictionary to loop through the data and create a dataframe for each river.
rivers = {
    'Guadalupe Rv nr Spring Branch': '08167500',
    'Guadalupe Rv at Sattler': '08167800',
    'Guadalupe Rv Abv Comal Rv at New Braunfels': '08168500',
    'San Marcos Rv at San Marcos': '08170500',
    'Blanco Rv at Wimberley': '08171000',
    'Medina Rv at Patterson Rd at Medina': '0817887350',
    'Medina Rv at La Coste': '08180640',
    'San Antonio Rv nr Floresville': '08183200',
}

# Convert the values in the dictionary to a list
rivers = list(rivers.values())

unified_rivers_df = nwis.get_record(sites=rivers, service='dv', start='2000-01-01', parameterCd='00060')

# Reset the index so that site_no is a column
unified_rivers_df = unified_rivers_df.reset_index()

# Create a new column called "River" that contains the name of the river based on the site number. Refer to the dictionary above to see which site number corresponds to which river.
unified_rivers_df['River'] = unified_rivers_df['site_no'].map({
    '08167500': 'Guadalupe Rv nr Spring Branch',
    '08167800': 'Guadalupe Rv at Sattler',
    '08168500': 'Guadalupe Rv Abv Comal Rv at New Braunfels',
    '08170500': 'San Marcos Rv at San Marcos',
    '08171000': 'Blanco Rv at Wimberley',
    '0817887350': 'Medina Rv at Patterson Rd at Medina',
    '08180640': 'Medina Rv at La Coste',
    '08183200': 'San Antonio Rv nr Floresville',
})

unified_rivers_df.to_csv('../output/rivers/unified_rivers.csv', index=False)

# Let's reorganize the columns so that the "River" column is the first column
unified_rivers_df = unified_rivers_df[['River', 'site_no', 'datetime', '00060_Mean']]

# Rename 00060_Maximum to "streamflow"
unified_rivers_df = unified_rivers_df.rename(columns={'00060_Mean': 'streamflow'})

unified_rivers_df.head()

,River,site_no,datetime,streamflow
0,Guadalupe Rv nr Spring Branch,08167500,2000-01-01 00:00:00+00:00,86.0
1,Guadalupe Rv nr Spring Branch,08167500,2000-01-02 00:00:00+00:00,89.0
2,Guadalupe Rv nr Spring Branch,08167500,2000-01-03 00:00:00+00:00,89.0
3,Guadalupe Rv nr Spring Branch,08167500,2000-01-04 00:00:00+00:00,86.0
4,Guadalupe Rv nr Spring Branch,08167500,2000-01-05 00:00:00+00:00,88.0


In [317]:
# Convert the datetime column to a datetime object
unified_rivers_df['datetime'] = pd.to_datetime(unified_rivers_df['datetime'])

# Create a new column called "Year" that contains the year of the datetime column
unified_rivers_df['Year'] = unified_rivers_df['datetime'].dt.year

# Let's create a new column titled "dw_date" that duplicates the data found in the "date" column, but replaces the year with 2050. We're doing all of this to make it easier for us to visualize it in a Datawrapper line chart in the future. The 2050 year will not appear in the final product and has no bearing on our analysis.
unified_rivers_df['dw_date'] = unified_rivers_df['datetime'].dt.strftime('%m/%d') + '/2050'

# Find the value in the "dw_date" column that is in the same row as the max value in the "date" column and print the first one out
latest_date = unified_rivers_df[unified_rivers_df['datetime'] == unified_rivers_df['datetime'].max()]['dw_date'].iloc[0]

# Let's filter the dataframe to only contain rows where the Year is greater than or equal to 2000
unified_rivers_df = unified_rivers_df[unified_rivers_df['Year'] >= 2000]

# Let's filter the dataframe to only include rows where the dw_date is between 2050-01-01 and 2050-03-20
unified_rivers_df = unified_rivers_df[(unified_rivers_df['dw_date'] >= '01/01/2050') & (unified_rivers_df['dw_date'] <= latest_date)].reset_index(drop=True)

# Break up the dataframe into separate csv files for each River
for river in unified_rivers_df['River'].unique():
    river_df = unified_rivers_df[unified_rivers_df['River'] == river]

    # Pivot the dataframe so that each year is a column and dw_date is the index with the values being the "streamflow" column. This is so that we can easily visualize it in a Datawrapper line chart.
    river_df = river_df.pivot(index='dw_date', columns='Year', values='streamflow').reset_index()
    # If a row has any missing values, drop it
    river_df = river_df.dropna()

    # Replace spaces with underscores in the file name
    river = river.replace(' ', '_')
    river_df.to_csv(f'../output/rivers/{river}.csv', index=False)

# unified_rivers_df.to_csv('../output/rivers/river_data.csv')
unified_rivers_df.head()

,River,site_no,datetime,streamflow,Year,dw_date
0,Guadalupe Rv nr Spring Branch,08167500,2000-01-01 00:00:00+00:00,86.0,2000,01/01/2050
1,Guadalupe Rv nr Spring Branch,08167500,2000-01-02 00:00:00+00:00,89.0,2000,01/02/2050
2,Guadalupe Rv nr Spring Branch,08167500,2000-01-03 00:00:00+00:00,89.0,2000,01/03/2050
3,Guadalupe Rv nr Spring Branch,08167500,2000-01-04 00:00:00+00:00,86.0,2000,01/04/2050
4,Guadalupe Rv nr Spring Branch,08167500,2000-01-05 00:00:00+00:00,88.0,2000,01/05/2050


## Playground

This section is for playing around with the data. It's not part of the collection of data, but it's useful for exploring.

In [319]:
# Find the median of each column. Put the results in a new dataframe
j17_historical_data_median_df = j17_historical_data_df.median().to_frame().reset_index()

# Sort the dataframe by the median values
j17_historical_data_median_df = j17_historical_data_median_df.sort_values(by=0, ascending=False).reset_index()

j17_historical_data_median_df

/var/folders/w4/djgt59l94t51k1198f2hnn980000gn/T/ipykernel_37495/4172157322.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  j17_historical_data_median_df = j17_historical_data_df.median().to_frame().reset_index()


,index,Year,0
0,5,2005,696.680
1,3,2003,693.470
2,8,2008,688.170
3,19,2019,686.340
4,17,2017,684.620
5,2,2002,682.250
6,4,2004,680.700
7,1,2001,679.980
8,6,2006,676.340
9,10,2010,676.040
